In [ ]:
from pgmpy.inference import BeliefPropagation, VariableElimination
def prob_pgmpy(model, target, evidence):
  """ Compute the probability of target given evidence
  """
  target_node, target_outcome = target
  belief_propagation = VariableElimination(model)
  result = belief_propagation.query(variables=[target_node], 
                                    evidence = evidence, 
                                    show_progress=True)
  p = result.get_value(**{target_node:target_outcome})
  return p

In [4]:
from explainbn.arguments import all_local_arguments
from explainbn.explanations import explain_argument
from pgmpy.readwrite import XMLBIFReader
reader = XMLBIFReader("spider.xml")
model = reader.get_model()

target = ('Spider','False')
evidence = {'Quinns':'False'}


# Find all arguments
arguments = all_local_arguments(model, target, evidence)

# Generate a textual explanation of each argument
for argument in arguments:
  explanation = explain_argument(model, argument)
  print(explanation)
  print("")

We have observed that Quinns is False.
That Quinns is False is evidence that Spider is True (weak 0.10008343850685929 inference).

We have observed that Quinns is False.
That Quinns is False is evidence that Both is False or Both is True (certain inf inference).
That Both is False or Both is True is evidence that Spider is False or Spider is True (certain inf inference).



In [ ]:
argument = arguments[4]
explanation = explain_argument(model, argument, mode='overview')
print(explanation)

In [ ]:
argument.nodes()

In [ ]:
argument.edges()

In [ ]:
argument.effects['Spider'].values

In [ ]:
expl = explain_argument(model, argument)
print(expl)

In [ ]:
#target = ('Both','True')
#evidence = {'Emersons': 'False','Quinns':'False', 'Sawyer': 'True', 'Winter': 'True', 'Alpha': 'True'}
p = prob_pgmpy(model,target,evidence)

In [ ]:
p

In [ ]:
argument = arguments[1]
explanation = explain_argument(model, argument, mode='overview')

from explainbn.explanations import explain_evidence
argument_effect = argument.effects[argument.target[0]]
conclusion_description, strength_qualifier = explain_evidence(model, argument_effect)

from explainbn.utilities import factor_to_outcomes
outcomes, strength = factor_to_outcomes(argument_effect, threshold=0.001)

In [ ]:
strength

In [ ]:
from functools import reduce
from itertools import combinations
import networkx as nx
import numpy as np
from explainbn.arguments import compute_argument_effects,all_simple_arguments,compute_argument_strength
from explainbn.utilities import (
  desextremize, 
  init_factor, 
  get_factor_from_scope, 
  factor_to_logodds, 
  to_factor_graph, 
  factor_distance, 
  make_argument_from_stack, 
  compose_arguments, 
  iterate_argument, 
  is_subargument, 
  limited_powerset, 
  partitions,
  random_evidence,
  random_outcome,
  )

target = ('Spider','True')
evidence = {'Emersons': 'False','Quinns': 'False'}

dependence_threshold = 0.1
path_length_limit = None
argument_complexity_limit = None

target_node, target_state = target

simple_arguments = \
all_simple_arguments(model, target, evidence, 
                     path_length_limit)

proper_arguments = {} # map of argument edge hashes 
                    # to arguments with cached effects

if argument_complexity_limit is None:
    argument_complexity_limit = len(simple_arguments)

for components in limited_powerset(simple_arguments, 
                                 argument_complexity_limit):
# Skip empty set of components
    if len(components) == 0: continue

# Compute the union of the components
    argument = compose_arguments(components)

# The union of components cannot contain loops
    if len(list(nx.simple_cycles(argument))) > 0:
        continue
    
    # Compute the effects of the union of components
    argument = compute_argument_effects(model, argument)
    total_effect = argument.effects[target_node]
    
    # Try to find a partition of the components
    # such that the effect of the total argument
    # equals the effect of the union of subarguments
    # formed by each group in the partition 
    components = list(components)
    for partition in partitions(components):
      
      # Skip trivial partition
        if len(partition) == 1: continue
    
        subarguments = [compose_arguments(subargument_components) 
                      for subargument_components in partition]
      
      # We retrieve the cached effects
        try:
            subarguments = \
          [proper_arguments[frozenset(subargument.edges)]
           for subargument in subarguments]
        except KeyError:
        # If a subargument is not in the cache is not proper.
        # We only need to check for combinations of 
        # proper subarguments
            continue
    
        subargument_effects = [subargument.effects[target_node] 
                              for subargument in subarguments]
    
        product_of_effects = reduce(lambda e1, e2 : e1*e2, 
                                  subargument_effects)
      
      # If the effect of the composite argument is the same as 
      # the subarguments then the argument is not proper
        if factor_distance(total_effect, product_of_effects) < \
           dependence_threshold:
            break
    else:
  # If no partition can emulate the effect of the argument
  # Add the argument to the set of proper arguments
        proper_arguments[frozenset(argument.edges)] = argument

arguments = proper_arguments.values()

# Filter non-maximal arguments
maximal_arguments = []
for arg1 in arguments:
    for arg2 in arguments:
        if arg1 == arg2: continue
        if is_subargument(arg1, arg2):
            break
    else:
        maximal_arguments.append(arg1)
arguments = maximal_arguments

# Combine non-independent pairs of arguments
# until all arguments are pairwise independent
refinement_is_possible = True
while refinement_is_possible:
    refinement_is_possible = False
    for arg1, arg2 in combinations(arguments,2):

        composite_argument = compose_arguments([arg1, arg2])

  # The union of components cannot contain loops
        if len(list(nx.simple_cycles(composite_argument))) > 0:
            continue

        composite_argument = \
    compute_argument_effects(model, composite_argument)
  
        try:
            composite_effect = composite_argument.effects[target_node]
        except KeyError:
            print([arg1, arg2, composite_argument])

        effect_product = \
    arg1.effects[target_node] * arg2.effects[target_node]

        if factor_distance(composite_effect, effect_product) > \
        dependence_threshold:
            arguments.remove(arg1)
            arguments.remove(arg2)
            arguments.append(composite_argument)
            refinement_is_possible = True
            break

# Order arguments by decreasing absolute strength
arguments.sort(key=lambda arg : 
                 abs(compute_argument_strength(model, arg)), 
             reverse=True)

In [ ]:
arguments[0].effects['Spider'].values

In [ ]:
arguments[3].effects['Spider'].values

In [ ]:
arguments[3].nodes()

In [ ]:
print("Nodes: ", model.nodes())
print("Edges: ", model.edges())
model.get_cpds()

In [ ]:
model.nodes().values()

In [ ]:
argument.effects['Spider']